In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()
SERVICE_KEY = os.getenv('OPEN_API_KEY')

function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}


llm = ChatOpenAI(
    openai_api_key=SERVICE_KEY,
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "rome"})


response = response.additional_kwargs["function_call"]["arguments"]

response

'{"questions":[{"question":"What year was Rome founded?","answers":[{"answer":"753 BC","correct":true},{"answer":"500 AD","correct":false},{"answer":"100 BC","correct":false}]},{"question":"Who was the first emperor of Rome?","answers":[{"answer":"Julius Caesar","correct":false},{"answer":"Augustus","correct":true},{"answer":"Nero","correct":false}]},{"question":"What event marked the fall of the Western Roman Empire?","answers":[{"answer":"The sack of Rome","correct":true},{"answer":"The Battle of Actium","correct":false},{"answer":"The Punic Wars","correct":false}]}]}'

In [2]:
import json

for question in json.loads(response)["questions"]:
    print(question)

{'question': 'What year was Rome founded?', 'answers': [{'answer': '753 BC', 'correct': True}, {'answer': '500 AD', 'correct': False}, {'answer': '100 BC', 'correct': False}]}
{'question': 'Who was the first emperor of Rome?', 'answers': [{'answer': 'Julius Caesar', 'correct': False}, {'answer': 'Augustus', 'correct': True}, {'answer': 'Nero', 'correct': False}]}
{'question': 'What event marked the fall of the Western Roman Empire?', 'answers': [{'answer': 'The sack of Rome', 'correct': True}, {'answer': 'The Battle of Actium', 'correct': False}, {'answer': 'The Punic Wars', 'correct': False}]}
